# 📊 ĐÁNH GIÁ VÀ CHUẨN HÓA DỮ LIỆU TĂNG CƯỜNG (DATA AUGMENTATION EVALUATION)

---

### 🎯 Mục tiêu
Notebook này thực hiện quy trình phân tích, so sánh và đánh giá chất lượng của các câu dữ liệu được sinh tự động (Data Augmentation) bởi hai mô hình ngôn ngữ lớn là **Gemini** và **ChatGPT** trong miền dữ liệu cụ thể (Coffee Shop). Mục đích là chọn lọc ra tập dữ liệu chất lượng cao nhất để huấn luyện mô hình dịch.

### 🛠️ Quy trình thực hiện (Pipeline)
1.  **Thống kê dữ liệu gốc:** Phân tích độ dài câu và từ vựng của tập dữ liệu `Coffee_Vie_VSL`
2.  **Tiền xử lý (Preprocessing):**
    * Loại bỏ các câu trùng lặp trong dữ liệu sinh bởi Gemini/ChatGPT.
    * Cân bằng số lượng mẫu (Downsampling) để đảm bảo tính công bằng khi so sánh.
3.  **Phân loại theo chủ đề:** Gán nhãn dữ liệu dựa trên từ điển menu (Cà phê, Trà, Sinh tố, Macchiato...).
4.  **Đánh giá chất lượng (Evaluation Metrics):**
    * **Type-Token Ratio (TTR):** Đo lường độ đa dạng của từ vựng.
    * **Cosine Similarity (TF-IDF):** Đo lường độ tương đồng ngữ nghĩa so với tập dữ liệu gốc.
5.  **Trực quan hóa & Xuất dữ liệu:** Vẽ biểu đồ so sánh và gộp dữ liệu sạch để chuẩn bị cho bước Training.

---

In [ ]:
!pip install pyvi sentence-transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## Thống kê số lượng từ

In [ ]:
base_path = '/content/drive/MyDrive/DUE - HỌC TẬP/YEAR 4 (2025-2026)/TTTN/raw_data/'
file_10k = base_path + 'Raw-Corpus-Vie-VSL-10K.csv'
file_coffee = base_path + 'Coffee_Vie_VSL.csv'
print(len(file_coffee))

88


In [ ]:
df = pd.read_csv(file_coffee)
print(len(df))

146


In [ ]:
def calculate_stats(file_path):
  df = pd.read_csv(file_path)

  col_vie = df.iloc[:, 0].astype(str)
  col_vsl = df.iloc[:, 1].astype(str)

  num_pairs = len(df)

  def get_vocab_size(text_series):
    all_text = " ".join(text_series.tolist())

    all_text = all_text.lower()
    tokens = all_text.split()

    vocab_set = set(tokens)

    return len(vocab_set)

  vocab_size_vie = get_vocab_size(col_vie)
  vocab_size_vsl = get_vocab_size(col_vsl)

  return num_pairs, vocab_size_vie, vocab_size_vsl

In [ ]:
stats_10k = calculate_stats(file_10k)
stats_coffee = calculate_stats(file_coffee)

In [ ]:
data = {
    "Chỉ số": ["Số lượng cặp câu", "Số lượng đơn vị từ vựng"],

    # Cột Vie-VSL-10k
    ("Vie-VSL-10k", "Số lượng trong câu tiếng Việt"): [stats_10k[0], stats_10k[1]],
    ("Vie-VSL-10k", "Số lượng trong câu VSL"):      [stats_10k[0], stats_10k[2]],

    # Cột Coffee-Vie-VSL
    ("Coffee-Vie-VSL", "Số lượng trong câu tiếng Việt"): [stats_coffee[0], stats_coffee[1]],
    ("Coffee-Vie-VSL", "Số lượng trong câu VSL"):      [stats_coffee[0], stats_coffee[2]]
}

# Tạo DataFrame với MultiIndex cho cột
df_result = pd.DataFrame(data)
df_result = df_result.set_index("Chỉ số")

# Hiển thị bảng
print("\n=== BẢNG THỐNG KÊ DỮ LIỆU ===")
display(df_result)


=== BẢNG THỐNG KÊ DỮ LIỆU ===


,"(Vie-VSL-10k, Số lượng trong câu tiếng Việt)","(Vie-VSL-10k, Số lượng trong câu VSL)","(Coffee-Vie-VSL, Số lượng trong câu tiếng Việt)","(Coffee-Vie-VSL, Số lượng trong câu VSL)"
Chỉ số,,,,
Số lượng cặp câu,10000,10000,145,145
Số lượng đơn vị từ vựng,3136,3148,136,53


In [ ]:
df = pd.read_csv(file_10k)

# Đảm bảo các cột được đọc là string (chuỗi)
# Tên cột dựa theo file ảnh đầu tiên bạn gửi
df['vietnamese'] = df['vietnamese'].astype(str)
df['vsl'] = df['vsl'].astype(str)

# 4. Đếm số lượng từ cho từng câu trong mỗi cột
# .str.split(): Tách câu thành một danh sách các từ (dựa trên khoảng trắng)
# .str.len(): Đếm số lượng phần tử trong danh sách đó
df['so_tu_vie'] = df['vietnamese'].str.split().str.len()
df['so_tu_vsl'] = df['vsl'].str.split().str.len()

# 5. Lọc ra những dòng có số từ Tiếng Việt > VSL
# Đây là bước chính để trả lời câu hỏi của bạn
df_ket_qua = df[df['so_tu_vie'] < df['so_tu_vsl']]

# 6. Hiển thị kết quả
print("\n=== KẾT QUẢ LỌC ===")
total_count = len(df_ket_qua)

if total_count > 0:
    print(f"Tìm thấy {total_count} câu có số lượng từ Tiếng Việt > VSL.")
    print("Hiển thị 21 ví dụ đầu tiên (hoặc tất cả nếu có ít hơn 21):")

    # Chỉ hiển thị các cột liên quan cho dễ nhìn
    columns_to_show = ['vietnamese', 'so_tu_vie', 'vsl', 'so_tu_vsl']
    display(df_ket_qua[columns_to_show].head(21))

    # In ra chỉ số (vị trí) của các dòng đó
    print("\nDanh sách chỉ số (câu thứ mấy) của các dòng trên là:")
    # df_ket_qua.index chứa danh sách các chỉ số dòng
    # (Lưu ý: Chỉ số bắt đầu từ 0)
    print(list(df_ket_qua.index))

else:
    print("Không tìm thấy câu nào có số từ Tiếng Việt nhiều hơn VSL.")


=== KẾT QUẢ LỌC ===
Tìm thấy 21 câu có số lượng từ Tiếng Việt > VSL.
Hiển thị 21 ví dụ đầu tiên (hoặc tất cả nếu có ít hơn 21):


,vietnamese,so_tu_vie,vsl,so_tu_vsl
12,Con gì biết bay ?,5,Động vật bay biết gì ?,6
91,Bạn thích bơi ở sông hay biển ?,8,Bạn thích bơi ở sông hay là ở biển ?,10
92,Tôi không thích màu cam và màu hồng,8,"Tôi màu cam , màu hồng không thích .",9
93,"Trong nhà , bạn thương ai nhất ?",8,Ở trong gia đình bạn thương nhất ai ?,9
120,Dưa hấu có vỏ xanh và ruột đỏ,8,Dưa hấu vỏ màu xanh bên trong màu đỏ .,10
148,Hằng ngắm mưa trong công viên .,7,Hằng mưa ngắm ở trong công viên .,8
355,Tôi mặc quần đùi,4,Tôi mặc quần đùi .,5
420,bạn có thể làm được,5,bạn có thể làm được .,6
464,Tôi đeo đồng hồ,4,Tôi đeo đồng hồ .,5
503,tại sao bạn hỏi,4,tại sao bạn hỏi .,5



Danh sách chỉ số (câu thứ mấy) của các dòng trên là:
[12, 91, 92, 93, 120, 148, 355, 420, 464, 503, 687, 688, 1019, 1231, 1236, 1459, 1519, 4349, 4363, 4492, 8139]


## Thống kê dữ liệu tăng cường

In [ ]:
# Menu dictionary
menu_dict = {
    "cà phê": ["cà phê sữa", "cà phê đen","cà phê đen phin", "cà phê đen máy", "cà phê đen Sài Gòn phin", "cà phê đen Sài Gòn máy", "bạc xỉu đá", "bạc xỉu nóng", "capuchino", "cà phê kem caramel", "cà phê muối", "cà phê dừa", "cacao đá", "cacao nóng", "socola đá", "socala nóng", "matcha latte", "cà phê moca", "cà phê trứng đá", "cà phê trứng nóng", "cacao muốn", "americano"],
    "macchiato": ["trà xoài macchiato", "trà xoài đậu biếc macchiato", "trà dâu đậu biếc macchiato", "trà dâu macchiato", "trà sen vàng macchiato", "milo macchiato", "matcha macchiato", "khoai môn macchiato", "socola macchiato"],
    "nước ép": ["nước chanh xí muội", "nước chanh hoa đậu biếc", "nước ép thơm", "nước ép cà rốt", "nước ép cà chua", "nước ép ổi", "nước ép cóc", "nước ép cam"],
    "sữa chua": ["sữa chua đào", "sữa chua dâu", "sữa chua xoài", "sữa chua cam nha đam", "sữa chua việt quất", "sữa chua kiwi nha đam", "sữa chua chanh dây"],
    "trà": ["trà gừng", "trà lipton cam thảo đá", "trà lipton cam thảo nóng", "trà đào cam sả đá", "trà đào cam sả nóng", "trà thạch sữa đào", "trà thạch vải", "trà vải cam sả", "trà cam nha đam", "trà cam quế mật ong đá","trà cam quế mật ong nóng", "trà olong dâu vải", "trà đào chanh leo"],
    "Món khác": ["sữa tươi trân châu đường đen","sữa tươi trân châu đường đen kem trứng", "sữa tươi trân châu đường đen trứng nướng","bò húc"],
    "đá xay": ["socola đá xay", "matcha đá xay", "cacao đá xay", "khoai môn đá xay", "cốt dừa cốm xanh", "bạc hà cookie đá xay"]
}

In [ ]:
def load_data(filename):
  path = base_path + filename
  df = pd.read_csv(path)
  df.columns = [c.lower().strip() for c in df.columns]
  text_data = df['vietnamese'].astype(str).str.lower().tolist()
  return text_data

def categorize_sentences(sentences, menu):
  categorized_data = {cat: [] for cat in menu.keys()}
  categorized_data['Không phân loại'] = []

  for sentence in sentences:
    found = False
    for category, items in menu.items():
      if any(item.lower() in sentence for item in items):
        categorized_data[category].append(sentence)
        found = True
      if not found:
        categorized_data['Không phân loại'].append(sentence)
  return categorized_data

def calculate_ttr(sentences):
  if not sentences: return 0

  all_text = " ".join(sentences)
  tokens = all_text.split()

  if len(tokens) == 0: return 0

  types = set(tokens)
  return len(types) / len(tokens)

def calculate_cosine_sim(original_docs, augmented_docs):
  if not original_docs or not augmented_docs:
    return 0

  combined_corpus = original_docs + augmented_docs
  vectorizer = TfidfVectorizer().fit(combined_corpus)

  # Biến đổi sang vector
  tfidf_original = vectorizer.transform(original_docs)
  tfidf_augmented = vectorizer.transform(augmented_docs)

  # Tính similarity của các câu Augmented so với TẤT CẢ câu Original
  # Kết quả là ma trận (n_aug x n_orig)
  cosine_sim_matrix = cosine_similarity(tfidf_augmented, tfidf_original)

  # Lấy giá trị trung bình của toàn bộ ma trận
  # Ý nghĩa: Trung bình mỗi câu augmented giống tập dữ liệu gốc đến mức nào
  return cosine_sim_matrix.mean()

In [ ]:
base_path = '/content/drive/MyDrive/DUE - HỌC TẬP/YEAR 4 (2025-2026)/TTTN/raw_data/'

In [ ]:
raw_coffee = load_data("Coffee_Vie_VSL.csv")
raw_gemini = load_data("Gemini_Coffee.csv")
raw_chatgpt = load_data("ChatGPT_Coffee.csv")

In [ ]:
print(f"Số lượng dòng ban đầu: Coffee={len(raw_coffee)}, Gemini={len(raw_gemini)}, ChatGPT={len(raw_chatgpt)}")

Số lượng dòng ban đầu: Coffee=146, Gemini=763, ChatGPT=1000


In [ ]:
# --- BƯỚC 2: Làm sạch trùng lặp trong tập tăng cường ---
print("\n--- 2. LOẠI BỎ TRÙNG LẶP (TRONG NỘI BỘ FILE) ---")
def remove_duplicates(data, name):
    unique_data = list(set(data))
    removed_count = len(data) - len(unique_data)
    print(f"- {name}: Đã loại bỏ {removed_count} câu trùng lặp. Còn lại: {len(unique_data)} câu.")
    return unique_data

clean_gemini = remove_duplicates(raw_gemini, "Gemini")
clean_chatgpt = remove_duplicates(raw_chatgpt, "ChatGPT")



--- 2. LOẠI BỎ TRÙNG LẶP (TRONG NỘI BỘ FILE) ---
- Gemini: Đã loại bỏ 218 câu trùng lặp. Còn lại: 545 câu.
- ChatGPT: Đã loại bỏ 0 câu trùng lặp. Còn lại: 1000 câu.


In [ ]:
# --- BƯỚC 3: Cân bằng dữ liệu (Downsampling ChatGPT) ---
print("\n--- 3. CÂN BẰNG DỮ LIỆU ĐỂ SO SÁNH ---")
target_count = len(clean_gemini)

if len(clean_chatgpt) > target_count:
    random.seed(42) # Cố định seed để kết quả không đổi
    sampled_chatgpt = random.sample(clean_chatgpt, target_count)
    print(f"Đã random trong ChatGPT từ {len(clean_chatgpt)} xuống còn {len(sampled_chatgpt)} câu (bằng số lượng Gemini).")
else:
    sampled_chatgpt = clean_chatgpt
    print(f"Số lượng ChatGPT ({len(clean_chatgpt)}) <= Gemini ({target_count}). Giữ nguyên.")


--- 3. CÂN BẰNG DỮ LIỆU ĐỂ SO SÁNH ---
Đã random trong ChatGPT từ 1000 xuống còn 545 câu (bằng số lượng Gemini).


In [ ]:
# --- BƯỚC 4: Phân loại theo Menu ---
# Chúng ta phân loại cả 3 tập dữ liệu
cat_coffee = categorize_sentences(raw_coffee, menu_dict)
cat_gemini = categorize_sentences(clean_gemini, menu_dict)
cat_chatgpt = categorize_sentences(sampled_chatgpt, menu_dict)

In [ ]:
# 4. THỐNG KÊ VÀ ĐÁNH GIÁ
# ==========================================

# --- Bảng 1: Thống kê số lượng (Yêu cầu 1) ---
stats_data = []
for category in menu_dict.keys():
    stats_data.append({
        "Danh mục": category,
        "Gốc (Câu)": len(cat_coffee[category]),
        "Gemini (Câu)": len(cat_gemini[category]),
        "ChatGPT (Câu)": len(cat_chatgpt[category])
    })

df_stats = pd.DataFrame(stats_data)
# Thêm hàng tổng cộng
df_stats.loc['Total'] = df_stats.sum(numeric_only=True)
df_stats.at['Total', 'Danh mục'] = 'TỔNG CỘNG'


# --- Bảng 2: Đánh giá chất lượng (Yêu cầu 2) ---
eval_data = []

for category in menu_dict.keys():
    # Lấy dữ liệu theo danh mục
    orig_texts = cat_coffee[category]
    gemini_texts = cat_gemini[category]
    gpt_texts = cat_chatgpt[category]

    # 1. Tính TTR (Độ đa dạng)
    ttr_orig = calculate_ttr(orig_texts)
    ttr_gemini = calculate_ttr(gemini_texts)
    ttr_gpt = calculate_ttr(gpt_texts)

    # 2. Tính Cosine Similarity (Độ tương đồng với tập gốc)
    # Lưu ý: Nếu danh mục đó không có câu nào trong tập gốc, không thể tính similarity
    sim_gemini = calculate_cosine_sim(orig_texts, gemini_texts)
    sim_gpt = calculate_cosine_sim(orig_texts, gpt_texts)

    eval_data.append({
        "Danh mục": category,
        "TTR Gốc": round(ttr_orig, 4),
        "TTR Gemini": round(ttr_gemini, 4),
        "TTR ChatGPT": round(ttr_gpt, 4),
        "Cosine Sim (Gemini vs Gốc)": round(sim_gemini, 4),
        "Cosine Sim (ChatGPT vs Gốc)": round(sim_gpt, 4)
    })

df_eval = pd.DataFrame(eval_data)

# ==========================================
# 5. HIỂN THỊ KẾT QUẢ
# ==========================================

print("\n" + "="*30)
print("BẢNG 1: THỐNG KÊ SỐ LƯỢNG CÂU THEO DANH MỤC MÓN")
print("="*30)
display(df_stats)

print("\n" + "="*30)
print("BẢNG 2: ĐÁNH GIÁ CHẤT LƯỢNG TĂNG CƯỜNG (TTR & COSINE SIMILARITY)")
print("="*30)
print("Ghi chú:")
print("- TTR (Type-Token Ratio): Càng cao -> Từ vựng càng phong phú/đa dạng.")
print("- Cosine Similarity: Càng gần 1 -> Ngữ nghĩa càng sát với tập dữ liệu gốc.")
display(df_eval)



BẢNG 1: THỐNG KÊ SỐ LƯỢNG CÂU THEO DANH MỤC MÓN


,Danh mục,Gốc (Câu),Gemini (Câu),ChatGPT (Câu)
0,cà phê,31.0,201.0,135.0
1,macchiato,0.0,101.0,8.0
2,nước ép,34.0,92.0,113.0
3,sữa chua,26.0,85.0,104.0
4,trà,10.0,48.0,110.0
5,Món khác,15.0,7.0,42.0
6,đá xay,0.0,12.0,20.0
Total,TỔNG CỘNG,116.0,546.0,532.0



BẢNG 2: ĐÁNH GIÁ CHẤT LƯỢNG TĂNG CƯỜNG (TTR & COSINE SIMILARITY)
Ghi chú:
- TTR (Type-Token Ratio): Càng cao -> Từ vựng càng phong phú/đa dạng.
- Cosine Similarity: Càng gần 1 -> Ngữ nghĩa càng sát với tập dữ liệu gốc.


,Danh mục,TTR Gốc,TTR Gemini,TTR ChatGPT,Cosine Sim (Gemini vs Gốc),Cosine Sim (ChatGPT vs Gốc)
0,cà phê,0.1541,0.0427,0.0835,0.1520,0.1834
1,macchiato,0.0000,0.0510,0.4154,0.0000,0.0000
2,nước ép,0.2069,0.0661,0.0922,0.1696,0.1798
3,sữa chua,0.2111,0.0628,0.0918,0.1859,0.1911
4,trà,0.3465,0.1257,0.0880,0.1700,0.1687
5,Món khác,0.2199,0.2877,0.1261,0.3383,0.2910
6,đá xay,0.0000,0.3049,0.2332,0.0000,0.0000


In [ ]:
import altair as alt

In [ ]:
# 1. Chuẩn bị dữ liệu
df_stats_plot = df_stats.drop('Total').reset_index(drop=True)
df_stats_long = df_stats_plot.melt(
    id_vars=['Danh mục'],
    value_vars=['Gốc (Câu)', 'Gemini (Câu)', 'ChatGPT (Câu)'],
    var_name='Nguồn dữ liệu',
    value_name='Số lượng câu'
)

In [ ]:
# 2. Vẽ biểu đồ 1
chart1 = alt.Chart(df_stats_long).mark_bar().encode(
    x=alt.X('Nguồn dữ liệu', axis=None),
    y=alt.Y('Số lượng câu', title='Số lượng câu'),
    color='Nguồn dữ liệu',
    column=alt.Column('Danh mục', header=alt.Header(titleOrient="bottom", labelOrient="bottom"), title='Danh mục món ăn'),
    tooltip=['Danh mục', 'Nguồn dữ liệu', 'Số lượng câu']
).properties(
    title='Biểu đồ 1: So sánh số lượng câu theo Danh mục'
).interactive()

In [ ]:
# BIỂU ĐỒ 2: SO SÁNH TTR (ĐỘ ĐA DẠNG) (từ df_eval)
# ==========================================================

# 1. Chuẩn bị dữ liệu TTR
df_ttr_long = df_eval.melt(
    id_vars=['Danh mục'],
    value_vars=['TTR Gốc', 'TTR Gemini', 'TTR ChatGPT'],
    var_name='Nguồn dữ liệu',
    value_name='TTR Score'
)

# 2. Vẽ biểu đồ 2
chart2 = alt.Chart(df_ttr_long).mark_bar().encode(
    x=alt.X('Nguồn dữ liệu', axis=None),
    y=alt.Y('TTR Score', title='TTR (Càng cao càng đa dạng)', scale=alt.Scale(domain=[0, 1])),
    color='Nguồn dữ liệu',
    column=alt.Column('Danh mục', header=alt.Header(titleOrient="bottom", labelOrient="bottom"), title='Danh mục món ăn'),
    tooltip=['Danh mục', 'Nguồn dữ liệu', 'TTR Score']
).properties(
    title='Biểu đồ 2: So sánh Độ đa dạng từ vựng (TTR)'
).interactive()


# ==========================================================
# BIỂU ĐỒ 3: SO SÁNH COSINE SIMILARITY (từ df_eval)
# ==========================================================

# 1. Chuẩn bị dữ liệu Cosine Sim
df_sim_long = df_eval.melt(
    id_vars=['Danh mục'],
    value_vars=['Cosine Sim (Gemini vs Gốc)', 'Cosine Sim (ChatGPT vs Gốc)'],
    var_name='Mô hình so sánh',
    value_name='Cosine Similarity'
)

# 2. Vẽ biểu đồ 3
chart3 = alt.Chart(df_sim_long).mark_bar().encode(
    x=alt.X('Mô hình so sánh', axis=None),
    y=alt.Y('Cosine Similarity', title='Cosine Sim (Càng gần 1 càng giống gốc)', scale=alt.Scale(domain=[0, 1])),
    color=alt.Color('Mô hình so sánh', legend=alt.Legend(title="So sánh với Gốc")),
    column=alt.Column('Danh mục', header=alt.Header(titleOrient="bottom", labelOrient="bottom"), title='Danh mục món ăn'),
    tooltip=['Danh mục', 'Mô hình so sánh', 'Cosine Similarity']
).properties(
    title='Biểu đồ 3: So sánh Độ tương đồng ngữ nghĩa (Cosine Similarity)'
).interactive()

In [ ]:
# HIỂN THỊ CẢ 3 BIỂU ĐỒ
print("--- HIỂN THỊ 3 BIỂU ĐỒ ---")
display(chart1)
display(chart2)
display(chart3)

--- HIỂN THỊ 3 BIỂU ĐỒ ---


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

## Tải file tăng cường về

In [ ]:
from google.colab import files

In [ ]:
def load_and_clean_df(filename, name, is_augmented=True):
    """
    Tải file CSV (cả 2 cột).
    Nếu là 'is_augmented=True', sẽ xóa trùng lặp dựa trên cột 'vietnamese'.
    Trả về một DataFrame đã xử lý.
    """
    try:
        path = base_path + filename
        df = pd.read_csv(path)
        # Chuẩn hóa tên cột
        df.columns = [c.lower().strip() for c in df.columns]

        # Đảm bảo có cột 'vietnamese'
        if 'vietnamese' not in df.columns:
            print(f"Lỗi: Không tìm thấy cột 'vietnamese' trong file {filename}")
            return pd.DataFrame()

        initial_count = len(df)

        if is_augmented:
            # Xóa các hàng trùng lặp, chỉ xét cột 'vietnamese'

            df_cleaned = df.drop_duplicates(subset=['vietnamese'], keep='first')

            final_count = len(df_cleaned)
            removed_count = initial_count - final_count
            print(f"- {name}: Đã loại bỏ {removed_count} câu trùng lặp (dựa trên 'vietnamese'). Còn lại: {final_count} câu.")
            return df_cleaned

        else:
            # Dữ liệu Gốc (Coffee) thì không xóa
            print(f"Đã tải {initial_count} câu Gốc ({name})")
            return df

    except Exception as e:
        print(f"Lỗi khi xử lý file {filename}: {e}")
        return pd.DataFrame() # Trả về DF rỗng nếu lỗi

In [ ]:
# --- 1. Chạy hàm để lấy DataFrames (DF) ---
# Tải Gốc (không xóa trùng lặp)
df_coffee = load_and_clean_df("Coffee_Vie_VSL.csv", "Gốc", is_augmented=False)

# Tải và làm sạch (xóa trùng lặp)
df_gemini = load_and_clean_df("Gemini_Coffee.csv", "Gemini", is_augmented=True)
df_chatgpt = load_and_clean_df("ChatGPT_Coffee.csv", "ChatGPT", is_augmented=True)

print("\n" + "="*30)
print("DỮ LIỆU ĐÃ SẴN SÀNG (DẠNG DATAFRAME)")
print(f"df_coffee (Gốc): {len(df_coffee)} dòng")
print(f"df_gemini (Đã làm sạch): {len(df_gemini)} dòng")
print(f"df_chatgpt (Đã làm sạch): {len(df_chatgpt)} dòng")

In [ ]:
df_combined = pd.concat([df_coffee, df_gemini, df_chatgpt], ignore_index = True)
output_file = 'Coffee_Combined.csv'
df_combined.to_csv(output_file, index=False, encoding = 'utf-8-sig')
files.download(output_file)